# Ноутбук 02 — Логика расчётов коэффициентов (коротко)

Здесь зафиксирована **схема данных**, **смысл числителя/знаменателя** и ссылки на **SQL-скрипты**, которые дают
результаты по AM и по отделу за каждый месяц 2023 и «итого за год».


## 1) Схема БД (MySQL)

Нужны две таблицы: **факт** помесячных оплат и **размерность** проекта (AM, последний месяц пролонгации).
Загрузка CSV — скриптами `load_chipments.sql` и `load_project_dim.sql` (путь к файлам и `secure-file-priv`).

```sql
CREATE DATABASE IF NOT EXISTS topface_media;
USE topface_media;

CREATE TABLE shipments (
  id VARCHAR(64),      -- проект
  month CHAR(7),       -- YYYY-MM
  shipment DECIMAL(15,2)
);

CREATE TABLE projects_dim (
  id VARCHAR(64),
  month_last CHAR(7),  -- YYYY-MM
  AM VARCHAR(255),
  Account VARCHAR(255)
);


## 2) Что считаем

**coef1 (1-й месяц)** — доля «сохранившихся» денег: берём проекты, которые **платили в M-1**, и смотрим их **платёж в M**.
**Формула M:** `coef1_M = SUM(num_M) / SUM(den_M)`, где `den_M = сумма платежей в (M-1)`, `num_M = сумма тех же id в M`.

**coef2 (2-й месяц)** — доля вернувшихся: берём проекты, которые **платили в M-2 и не платили в M-1**, и смотрим **платёж в M**.
**Формула M:** `coef2_M = SUM(num2_M) / SUM(den2_M)`.

Срезы: **AM** и **Отдел**. Период: **2023-01…2023-12** + **итого за 2023**.

## 3) Логика coef1

**Знаменатель den_M:** суммы оплат за **(M-1)**, промаркированы как **M**.
**Числитель num_M:** суммы оплат в **M** по тем же id (у кого была оплата в **M-1**).
**Итог:** `coef1_M = SUM(num_M) / SUM(den_M)`.

```sql
-- Скелет запроса coef1 по AM × месяцам
WITH prev AS (  -- (M-1) -> маркируем как M
  SELECT s.id,
         DATE_FORMAT(DATE_ADD(STR_TO_DATE(CONCAT(s.month,'-01'),'%Y-%m-%d'),INTERVAL 1 MONTH),'%Y-%m') AS month,
         s.shipment AS den
  FROM shipments s
),
cur AS (        -- суммы в M
  SELECT id, month, shipment AS num
  FROM shipments
),
joined AS (
  SELECT p.month, COALESCE(NULLIF(TRIM(d.AM),''),'Без АМ') AS AM,
         SUM(c.num) AS num, SUM(p.den) AS den
  FROM prev p
  JOIN cur c ON c.id=p.id AND c.month=p.month
  JOIN projects_dim d ON d.id=p.id
  WHERE p.month BETWEEN '2023-01' AND '2023-12'
  GROUP BY p.month, AM
)
SELECT month, AM, num, den, num/NULLIF(den,0) AS coef1
FROM joined;


## 4) Логика coef2

**База id:** платили в **(M-2)** и **не** платили в **(M-1)**.
**Знаменатель den2_M:** суммы **(M-2)**, промаркированы как **M**.
**Числитель num2_M:** суммы в **M** по тем же id.
**Итог:** `coef2_M = SUM(num2_M) / SUM(den2_M)`.

```sql
WITH pay_m2 AS (  -- (M-2) -> маркируем как M
  SELECT s.id,
         DATE_FORMAT(DATE_ADD(STR_TO_DATE(CONCAT(s.month,'-01'),'%Y-%m-%d'),INTERVAL 2 MONTH),'%Y-%m') AS month,
         s.shipment AS den2
  FROM shipments s
),
pay_m1 AS (      -- (M-1)
  SELECT s.id,
         DATE_FORMAT(DATE_ADD(STR_TO_DATE(CONCAT(s.month,'-01'),'%Y-%m-%d'),INTERVAL 1 MONTH),'%Y-%m') AS month
  FROM shipments s
),
base AS (        -- есть (M-2), нет (M-1)
  SELECT m2.id, m2.month, m2.den2
  FROM pay_m2 m2
  LEFT JOIN pay_m1 m1 ON m1.id=m2.id AND m1.month=m2.month
  WHERE m1.id IS NULL
)
SELECT b.month, COALESCE(NULLIF(TRIM(d.AM),''),'Без АМ') AS AM,
       SUM(cur.shipment) AS num2, SUM(b.den2) AS den2,
       SUM(cur.shipment)/NULLIF(SUM(b.den2),0) AS coef2
FROM base b
LEFT JOIN shipments cur ON cur.id=b.id AND cur.month=b.month
JOIN projects_dim d ON d.id=b.id
WHERE b.month BETWEEN '2023-01' AND '2023-12'
GROUP BY b.month, AM;

## 5) Технические примечания

Нормализация AM: `TRIM` + пустые → **«Без АМ»**.
Фильтр периода: **2023-01…2023-12**.
Сдвиги месяцев: `DATE_FORMAT(CONCAT(month,'-01') ± INTERVAL 1/2 MONTH, '%Y-%m')`.
Готовые выгрузки — в `sql_results/*.csv`.

## 6) Что на выходе

Форматы выгрузок: `month`, `AM` (или `department`), `num/den` или `num2/den2`, `coef1/coef2`.
Итоговый отчёт: `report/report.xlsx` (лист **Dashboard** и таблицы `coef1_*`, `coef2_*`).


## 7) Как воспроизвести

**01:** подготовить `out/shipments.csv`, `out/projects_dim.csv` (см. ноутбук 01).
**02:** загрузить в MySQL (`load_chipments.sql`, `load_project_dim.sql`).
**03:** выполнить SQL из `sql_requests/` (`coef1_*`, `coef2_*`).
**04:** сохранить результаты в `sql_results/*.csv`.
